In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
%reload_ext autoreload

In [25]:
import json

with open('podcast.txt', 'rb') as f:
    request = json.load(f)

In [94]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-07-04T12:15:14Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")


In [91]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
import json
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_pims_score(req):

    #if req_data is None:
    #    lambda_payload = {"body": input_list}
    #    print (json.dumps(lambda_payload))
    #else:
    #    lambda_payload = {"body": {"request": req_data, "text_input": input_list}}
        
    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="pim",
            InvocationType="RequestResponse",
            Payload=json.dumps(req),
        )
        lambda_output = (
            invoke_response["Payload"].read().decode("utf8")
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            result = json.loads(response_body)['d2vResult'][0]['distance']
        return result
    except Exception as e:
        print (e)
    return False

In [92]:
pim_result = {}
pim_request = {"contextId": request["body"]["contextId"], "mindId": "01daaqy88qzb19jqz5prjfr76y"}
for seg in request['body']['segments']:
    pim_request["segments"] = [seg]
    # get_pims_score({"body":pim_request})
    pim_result[seg["recordingId"]] =  get_pims_score({"body":pim_request})

In [95]:
group

{'group': {'15': [{'id': '1bc499f637aa41779390460a565e4524',
    'originalText': 'So you can see the that areas you have five alias mentioned here.',
    'confidence': 0.9471649,
    'startTime': '2019-09-22T09:54:05Z',
    'endTime': '2019-09-22T09:54:12Z',
    'duration': 7,
    'recordingId': '39705b73c1074b96822bf145c23e7704',
    'spokenBy': '31a3ba4761854ad9a041ddf1c4c6a1dc',
    'languageCode': 'en-IN',
    'transcriber': 'deepgram',
    'status': 'completed',
    'transcriptId': '86dfd87a-6a7d-4164-ba01-a40a24c8daef',
    'createdAt': '2019-09-22T09:54:12.942977446Z',
    'updatedAt': '2019-09-22T09:54:14.715094811Z',
    'deletedAt': None,
    'deleted': False}],
  '24': [{'id': '8ad2cdf53c0c47ee97ecc9eb649af405',
    'originalText': "Ux Designers have a a lot on their plate. They research that wire user tests the list goes on and off if you're new to the game this can all feel very overwhelming well throw that here out the window because we're about to embark on a journey thr

In [96]:
topic_pim = {
}
group_result = {}
for keys in group['group'].keys():
    for seg in group['group'][keys]:
        group_result[seg['recordingId']] = keys

In [118]:
ranked_pims = sorted([(k,v) for (k,v) in pim_result.items()], key= lambda kv: kv[1])

In [119]:
used_topics = []
group_no = None
index = 0
for (rec_id, distance) in ranked_pims:
    if rec_id in group_result.keys():
        group_no = group_result[rec_id]
        
        if group_no not in used_topics:
            topic_pim[index] = group_no
            used_topics.append(group_no)
            index += 1

In [120]:
topic_pim

{0: '24', 1: '40', 2: '32', 3: '15'}

In [125]:
final_output = []
final_output = list(map(lambda x: group['group'][x] , topic_pim.values()))

In [126]:
final_output[0][0]

{'id': '8ad2cdf53c0c47ee97ecc9eb649af405',
 'originalText': "Ux Designers have a a lot on their plate. They research that wire user tests the list goes on and off if you're new to the game this can all feel very overwhelming well throw that here out the window because we're about to embark on a journey through the sixteenth end of the UX design process that before starting any successful uX project you'll need to be in tune with two things your user in your brain. I start with the user big question around this is what problems are they making once you can identify the problem, then you can begin to start working to towards the solution second is understanding the brain and how does the project you're are working on aligned with the branch mission and goal to summarize in this stage you wanna identify the problems facing the users as well as the core goals of the brand and see how they along as a designer you probably have a gun of a*sumptions about what the user wants or doesn't want b

In [129]:
users = []
for result in final_output:
    temp_users = []
    for seg in result:
        if seg['spokenBy'] not in temp_users:
            temp_users.append(seg['spokenBy'])
    users.append(temp_users)

In [130]:
users

[['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc']]